<a href="https://colab.research.google.com/github/saranpan/guilding-nb/blob/main/thai_tokenization_evolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we are going to start from the simpliest to the most complicated but efficient

Status:

✅ Done
❌ Not yet

# ✅ Greedy First vocab match

In [ ]:
# Predefined Vocab

vocab = ['กิน','ข้าว','อร่อย', 'ผม']

In [ ]:
test_txt = 'ผมกินข้าว'

ถ้าไม่เจอ token ที่อยู่ใน vocab จะให้เป็น `<unk>` แทน

In [ ]:
vocab = ['กิน','ข้าว','อร่อย', 'ผม']
test_txt = 'ผมกินข้าว'

token_lst = []
skip_ = 1
i = 0

while i < len(test_txt):  # i : start from word position i_th

    for j in range(len(test_txt)-i): # j : try from 0 until the position which match with vocab
        
        txt = test_txt[i:i+j+1]
        print(txt)
        if txt in vocab:
            token_lst.append(txt)
            skip_ = len(txt)
            print(f'{txt} : found in vocab')
            i += skip_
            break
        else:
            print(f'{txt} : NOT found in vocab')
            
        if j == len(test_txt)-i:  # Match until the last character and still does not form any of tokens
            token_lst.append('<UNK>')
            skip_ = len(txt)
            print('=== Cannot detected ===')
            i += skip_
            break

ผ
ผ : NOT found in vocab
ผม
ผม : found in vocab
ก
ก : NOT found in vocab
กิ
กิ : NOT found in vocab
กิน
กิน : found in vocab
ข
ข : NOT found in vocab
ข้
ข้ : NOT found in vocab
ข้า
ข้า : NOT found in vocab
ข้าว
ข้าว : found in vocab


In [ ]:
token_lst

['ผม', 'กิน', 'ข้าว']

In [ ]:
# ลองมา form มาให้เป็น defined function เพื่อ reusable

def greedy_match_tokenizer(test_txt,vocab):

    token_lst = []
    skip_ = 1
    i = 0

    while i < len(test_txt):  # i : start from word position i_th
        
        for j in range(len(test_txt)-i): # j : try from 0 until the position which match with vocab
            txt = test_txt[i:i+j+1]

            if txt in vocab:
                token_lst.append(txt)
                skip_ = len(txt)
                print(f'{txt} : found in vocab \n')
                i += skip_
                break
            else:
                print(f'{txt} : NOT found in vocab')

            if j == len(test_txt)-i-1:  # Match until the last character and still does not form any of tokens
                token_lst.append('<UNK>')
                skip_ = len(txt)
                print('=== Cannot detected ===')
                i += skip_
                break
            

    return token_lst

In [ ]:
vocab = ['กิน','ข้าว','อร่อย', 'ผม']
test_txt = 'ผมกินข้าว'

greedy_match_tokenizer(test_txt,vocab)

ผ : NOT found in vocab
ผม : found in vocab 

ก : NOT found in vocab
กิ : NOT found in vocab
กิน : found in vocab 

ข : NOT found in vocab
ข้ : NOT found in vocab
ข้า : NOT found in vocab
ข้าว : found in vocab 



['ผม', 'กิน', 'ข้าว']

In [ ]:
vocab = ['ฉัน','สยาม','ผม','พารากอน', 'นั่ง','รถ','รถไฟฟ้า','ไฟฟ้า', 'ไป','เมื่อวาน']
test_txt = 'ผมนั่งรถไฟฟ้าไปสยามพารากอนเมื่อวาน'

greedy_match_tokenizer(test_txt,vocab)

ผ : NOT found in vocab
ผม : found in vocab 

น : NOT found in vocab
นั : NOT found in vocab
นั่ : NOT found in vocab
นั่ง : found in vocab 

ร : NOT found in vocab
รถ : found in vocab 

ไ : NOT found in vocab
ไฟ : NOT found in vocab
ไฟฟ : NOT found in vocab
ไฟฟ้ : NOT found in vocab
ไฟฟ้า : found in vocab 

ไ : NOT found in vocab
ไป : found in vocab 

ส : NOT found in vocab
สย : NOT found in vocab
สยา : NOT found in vocab
สยาม : found in vocab 

พ : NOT found in vocab
พา : NOT found in vocab
พาร : NOT found in vocab
พารา : NOT found in vocab
พาราก : NOT found in vocab
พารากอ : NOT found in vocab
พารากอน : found in vocab 

เ : NOT found in vocab
เม : NOT found in vocab
เมื : NOT found in vocab
เมื่ : NOT found in vocab
เมื่อ : NOT found in vocab
เมื่อว : NOT found in vocab
เมื่อวา : NOT found in vocab
เมื่อวาน : found in vocab 



['ผม', 'นั่ง', 'รถ', 'ไฟฟ้า', 'ไป', 'สยาม', 'พารากอน', 'เมื่อวาน']

More Practical vocab : เนื่องจาก vocab ที่เราโชว์ให้ดูก่อนหน้าเป็นแค่ส่งนนึงของ vocab จริงๆ เร็วๆนี้เราจะใช้ vocab ที่มาจาก pyThainlp ที่รวบรวมทุกคำมา

Main Drawback : 

- sensitive กับคำสะกดผิด เช่น กิรข้าวยัง จะไม่สามารถ detect anything (ลามยาวไปถึงคำสุดท้าย)

- vocab ต้องอัพเหตด้วยมือตลอดเวลาเนื่องด้วยความไม่เสถียรของภาษาวิบัติที่เปลี่ยนแปลงตลอดเวลา เช่น จุงเบย, ขก, สายมู (sol แบบ หยาบๆ: เพิ่มvocabพวกนั้น)

- เนื่องด้วยการ adopt greedy search algorithm มาทำให้ไม่สามารถ detect คำแบบ 'น่าร้าก' , 'น่าร๊อก' เพราะคำว่า 'น่า' ก็เป็นคำที่มีความหมาย

- หากมีคำอังกถษคำไทยคำ vocab size จะต้องเยอะขึ้น และ เราต้องแก้ไขปัญหาพวกนี้ทั้งไทยทั้งอังกถษ

In [ ]:
vocab = ['เธอ','เก่ง','จัง','เลย','กะเพรา']
test_txt = 'เทอเก่งจุงเบย'

greedy_match_tokenizer(test_txt,vocab)

เ : NOT found in vocab
เท : NOT found in vocab
เทอ : NOT found in vocab
เทอเ : NOT found in vocab
เทอเก : NOT found in vocab
เทอเก่ : NOT found in vocab
เทอเก่ง : NOT found in vocab
เทอเก่งจ : NOT found in vocab
เทอเก่งจุ : NOT found in vocab
เทอเก่งจุง : NOT found in vocab
เทอเก่งจุงเ : NOT found in vocab
เทอเก่งจุงเบ : NOT found in vocab
เทอเก่งจุงเบย : NOT found in vocab
=== Cannot detected ===


['<UNK>']

In [ ]:
vocab = ['เธอ','น่า','น่ากิน','น่ารัก','น่าเล่น','รัก']
test_txt = 'เธอน่ารัก'

greedy_match_tokenizer(test_txt,vocab)
# Expected token_lst : ['เธอ','น่ารัก']

เ : NOT found in vocab
เธ : NOT found in vocab
เธอ : found in vocab 

น : NOT found in vocab
น่ : NOT found in vocab
น่า : found in vocab 

ร : NOT found in vocab
รั : NOT found in vocab
รัก : found in vocab 



['เธอ', 'น่า', 'รัก']

In [ ]:
vocab = ['กะเพรา','น่ากิน','น่ารัก','น่าเล่น','รัก','เลย','มาก']
test_txt = 'กะเพราน่้ากินมากเลย'

greedy_match_tokenizer(test_txt,vocab)
# Expected token_lst : ['กะเพรา','น่้า','กิน','มาก','เลย'] 

ก : NOT found in vocab
กะ : NOT found in vocab
กะเ : NOT found in vocab
กะเพ : NOT found in vocab
กะเพร : NOT found in vocab
กะเพรา : found in vocab 

น : NOT found in vocab
น่ : NOT found in vocab
น่้ : NOT found in vocab
น่้า : NOT found in vocab
น่้าก : NOT found in vocab
น่้ากิ : NOT found in vocab
น่้ากิน : NOT found in vocab
น่้ากินม : NOT found in vocab
น่้ากินมา : NOT found in vocab
น่้ากินมาก : NOT found in vocab
น่้ากินมากเ : NOT found in vocab
น่้ากินมากเล : NOT found in vocab
น่้ากินมากเลย : NOT found in vocab
=== Cannot detected ===


['กะเพรา', '<UNK>']

Anyway, ในโลกความเป็นจริง vocab size ควรมากกว่านี้, เราสามารถโหลด vocab จำนวนใหญ่ได้ใน package pythainlp



In [ ]:
!pip -q install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 36.0 MB/s eta 0:00:00


In [ ]:
from pythainlp.corpus.common import thai_words
vocab = list(thai_words())
print(f'vocab size : {len(vocab)}')

vocab size : 62055


In [ ]:
# มาดู 10 คำแรกกัน

vocab[:10]

['บัณฑรนาค',
 'สามสิบกลีบ',
 'ดุสิตรีสอร์ท',
 'ประสูต',
 'หัตถาโรหะ',
 'เหมือนว่า',
 'ค่าหด',
 'ชุษณปักษ์',
 'อวเคราะห์',
 'ราตรี']

In [ ]:
# มาลองกับ test_txt อะไรก็ได้ของเรากัน

test_txt = 'ผมว่าผมต้องกินข้าวผัดทอดร้อนๆที่กรุงเทพมหานคร'

greedy_match_tokenizer(test_txt,vocab)

ผ : NOT found in vocab
ผม : found in vocab 

ว : NOT found in vocab
ว่ : NOT found in vocab
ว่า : found in vocab 

ผ : NOT found in vocab
ผม : found in vocab 

ต : NOT found in vocab
ต้ : NOT found in vocab
ต้อ : found in vocab 

ง : NOT found in vocab
งก : found in vocab 

ิ : NOT found in vocab
ิน : NOT found in vocab
ินข : NOT found in vocab
ินข้ : NOT found in vocab
ินข้า : NOT found in vocab
ินข้าว : NOT found in vocab
ินข้าวผ : NOT found in vocab
ินข้าวผั : NOT found in vocab
ินข้าวผัด : NOT found in vocab
ินข้าวผัดท : NOT found in vocab
ินข้าวผัดทอ : NOT found in vocab
ินข้าวผัดทอด : NOT found in vocab
ินข้าวผัดทอดร : NOT found in vocab
ินข้าวผัดทอดร้ : NOT found in vocab
ินข้าวผัดทอดร้อ : NOT found in vocab
ินข้าวผัดทอดร้อน : NOT found in vocab
ินข้าวผัดทอดร้อนๆ : NOT found in vocab
ินข้าวผัดทอดร้อนๆท : NOT found in vocab
ินข้าวผัดทอดร้อนๆที : NOT found in vocab
ินข้าวผัดทอดร้อนๆที่ : NOT found in vocab
ินข้าวผัดทอดร้อนๆที่ก : NOT found in vocab
ินข้าวผัดทอดร้อนๆที่กร : NOT fou

['ผม', 'ว่า', 'ผม', 'ต้อ', 'งก', '<UNK>']

จะเห็นว่า algorithm นี้ไม่ดีเนื่องจากข้อเสียจาก greedy search algorithm



---



# ✅ Maximal First vocab match (Maximal Matching)

Similar to greedy first vocabulary match, but instead of splitting the token upon detection, it continues until the longest word is found. The first step is trying starting with first character will attempt to slice until the last character.

The possible worst case of time complexity would be when every token contain 1 characters (time complexity = O(N!) )

ref : https://medium.com/@anshul16/maximum-matching-word-segmentation-algorithm-python-code-3444fe4bd6f9

In [ ]:
from pythainlp.corpus.common import thai_words
vocab = set(thai_words()) # Use set is more practical than list to avoid duplicate vocab

มาลองคำนี้ดีกว่า 'หมาป่าน่ารักมาก' <br>
เรา expect ว่า ['หมาป่า','น่ารัก','มาก']


In [ ]:
test_txt = 'หมาป่าน่ารักมาก'

In [ ]:
# ถ้าลอง algorithm แรก จะไม่ได้ตามต้องการ

greedy_match_tokenizer(test_txt, vocab)

ห : NOT found in vocab
หม : NOT found in vocab
หมา : found in vocab 

ป : NOT found in vocab
ป่ : NOT found in vocab
ป่า : found in vocab 

น : NOT found in vocab
น่ : NOT found in vocab
น่า : found in vocab 

ร : NOT found in vocab
รั : NOT found in vocab
รัก : found in vocab 

ม : NOT found in vocab
มา : found in vocab 

ก : NOT found in vocab
=== Cannot detected ===


['หมา', 'ป่า', 'น่า', 'รัก', 'มา', '<UNK>']

In [ ]:
# มาลองสร้าง maximal matching กัน

token_lst = []

skip_ = 1
i = 0

while i < len(test_txt):  # i : start from word position i_th

    current_max_word = None

    for j in range(len(test_txt)-i): # j : try from 0 until the position which match with vocab
        
        txt = test_txt[i:i+j+1]
        if txt in vocab:
            """เปลี่ยนนิดหน่อย จาก algorithm ของ greedy match
            1. เราจะยังไม่ append ทันที จนกว่าจะ slice ไปถึง last character
            2. ไม่ break หลัง detect คำ
            """
            current_max_word = txt
            skip_ = len(current_max_word)
            print(f'{txt} : found in vocab')
        else:
            print(f'{txt} : not found in vocab')
            

    if current_max_word:
        print(f'Last detected word -> {current_max_word}  \n\n')
        token_lst.append(current_max_word)
        i += skip_
        
    else:
        token_lst.append('<unk>')
        break


ห : not found in vocab
หม : not found in vocab
หมา : found in vocab
หมาป : not found in vocab
หมาป่ : not found in vocab
หมาป่า : found in vocab
หมาป่าน : not found in vocab
หมาป่าน่ : not found in vocab
หมาป่าน่า : not found in vocab
หมาป่าน่าร : not found in vocab
หมาป่าน่ารั : not found in vocab
หมาป่าน่ารัก : not found in vocab
หมาป่าน่ารักม : not found in vocab
หมาป่าน่ารักมา : not found in vocab
หมาป่าน่ารักมาก : not found in vocab
Last detected word -> หมาป่า  


น : not found in vocab
น่ : not found in vocab
น่า : found in vocab
น่าร : not found in vocab
น่ารั : not found in vocab
น่ารัก : found in vocab
น่ารักม : not found in vocab
น่ารักมา : not found in vocab
น่ารักมาก : not found in vocab
Last detected word -> น่ารัก  


ม : not found in vocab
มา : found in vocab
มาก : found in vocab
Last detected word -> มาก  




In [ ]:
token_lst

['หมาป่า', 'น่ารัก', 'มาก']

มา form เป็น defined function กันเลย 

In [ ]:
def maximal_match_tokenizer(test_txt,vocab):
    token_lst = []

    skip_ = 1
    i = 0

    while i < len(test_txt):  # i : start from word position i_th

        current_max_word = None

        for j in range(len(test_txt)-i): # j : try from 0 until the position which match with vocab
            
            txt = test_txt[i:i+j+1]
            if txt in vocab:
                """เปลี่ยนนิดหน่อย จาก algorithm ของ greedy match
                1. เราจะยังไม่ append ทันที จนกว่าจะ slice ไปถึง last character
                2. ไม่ break หลัง detect คำ
                """
                current_max_word = txt
                skip_ = len(current_max_word)
                print(f'{txt} : found in vocab')
            else:
                print(f'{txt} : not found in vocab')
                

        if current_max_word:
            print(f'Last detected word -> {current_max_word}  \n\n')
            token_lst.append(current_max_word)
            i += skip_
            
        else:
            token_lst.append('<unk>')
            break

    return token_lst

In [ ]:
vocab = set(thai_words()) 
test_txt = 'หมาป่าน่ารักมาก'
maximal_match_tokenizer(test_txt,vocab)

ห : not found in vocab
หม : not found in vocab
หมา : found in vocab
หมาป : not found in vocab
หมาป่ : not found in vocab
หมาป่า : found in vocab
หมาป่าน : not found in vocab
หมาป่าน่ : not found in vocab
หมาป่าน่า : not found in vocab
หมาป่าน่าร : not found in vocab
หมาป่าน่ารั : not found in vocab
หมาป่าน่ารัก : not found in vocab
หมาป่าน่ารักม : not found in vocab
หมาป่าน่ารักมา : not found in vocab
หมาป่าน่ารักมาก : not found in vocab
Last detected word -> หมาป่า  


น : not found in vocab
น่ : not found in vocab
น่า : found in vocab
น่าร : not found in vocab
น่ารั : not found in vocab
น่ารัก : found in vocab
น่ารักม : not found in vocab
น่ารักมา : not found in vocab
น่ารักมาก : not found in vocab
Last detected word -> น่ารัก  


ม : not found in vocab
มา : found in vocab
มาก : found in vocab
Last detected word -> มาก  




['หมาป่า', 'น่ารัก', 'มาก']

In [ ]:
test_txt = 'หมอดินกินไรนกฟิ'
maximal_match_tokenizer(test_txt,vocab)

ห : not found in vocab
หม : not found in vocab
หมอ : found in vocab
หมอด : not found in vocab
หมอดิ : not found in vocab
หมอดิน : not found in vocab
หมอดินก : not found in vocab
หมอดินกิ : not found in vocab
หมอดินกิน : not found in vocab
หมอดินกินไ : not found in vocab
หมอดินกินไร : not found in vocab
หมอดินกินไรน : not found in vocab
หมอดินกินไรนก : not found in vocab
หมอดินกินไรนกฟ : not found in vocab
หมอดินกินไรนกฟิ : not found in vocab
Last detected word -> หมอ  


ด : not found in vocab
ดิ : found in vocab
ดิน : found in vocab
ดินก : not found in vocab
ดินกิ : not found in vocab
ดินกิน : not found in vocab
ดินกินไ : not found in vocab
ดินกินไร : not found in vocab
ดินกินไรน : not found in vocab
ดินกินไรนก : not found in vocab
ดินกินไรนกฟ : not found in vocab
ดินกินไรนกฟิ : not found in vocab
Last detected word -> ดิน  


ก : not found in vocab
กิ : not found in vocab
กิน : found in vocab
กินไ : not found in vocab
กินไร : not found in vocab
กินไรน : not found in vocab
กินไรนก : n

['หมอ', 'ดิน', 'กิน', 'ไร', 'นก', '<unk>']

Main Drawback:
1. algorithm เชื่อว่า token ที่ดีที่สุดคือ token ที่ต้องยาวที่สุดเท่าที่เป็นไปได้ แต่ในความเป็นจริง มันไม่จำเป็นต้องเป็นนั้นเสมอไป แล้วเราจะรู้ได้ไงหละ ว่า token ไหนดีสุด ด้วยเหตุนี้เราต้องใช้ Probability Theory มาเพื่อวัด score ของแต่ token candidate

2. ใช้เวลานานหาก token ที่พบมันสั้นมาก 

# ❌ N-gram model 

ปัญหาท่เจอก่อนหน้านี้จาก maximal matching คือ การ assume ว่า token ยิ่งยาวยิ่งดีกว่าเสมอ ซึ่งไม่จริงเสมอไป

การที่เราจะแยกได้ว่าอะไรดีกว่าอาจจะต้องมีการ annotate score ของแต่ละ token ซึ่ง score ก็จะวัดจาก frequency ที่พบใน corpus (ที่เราเก็บมา)


หากเราทำดังกล่าวก็จะเป็น Unigram

แต่หากเราทำโดยให้ score ของ token โดยดูว่ามีโอกาสเกิดขึ้นกับ token ก่อนหน้าไหม ก็จะได้อีก score นึง 

In [ ]:
# Unigram vocab

vocab_unigram = {'เท' : 180 ,'กิน' : 300, 
                 'กินูริ' : 1 , 'ข้า' : 10,
                 'ยัง' : 150,
                 'เทอ' : 200}

algorithm จะทำเหมือนกับ maximal matching แต่จะเก็บทุก candidate ที่ found ใน vocab และเลือกอันที่ score มากที่สุด

In [ ]:
# มาลอง test คำว่า 'เทอกินข้าวยัง' # 
test_txt = 'เทอกินข้าวยัง'

In [ ]:
# This is not UNIGRAM since we have to try both เท and เทอ 

token_lst = []

skip_ = 1
i = 0

while i < len(test_txt):  # i : start from word position i_th

    current_max_word   = None
    max_p_txt = 0
    for j in range(len(test_txt)-i): # j : try from 0 until the position which match with vocab
        
        txt = test_txt[i:i+j+1]
        if txt in vocab_unigram:
            """เปลี่ยนนิดหน่อย จาก algorithm ของ maximal match
            1. append ทุกๆ candidate ที่ found in vocab
            """
            p_txt = vocab_unigram.get(txt)
            if p_txt > max_p_txt:
                current_max_word = txt
                max_p_txt = p_txt
            
            skip_ = len(current_max_word)
            print(f'{txt} : found in vocab')
        else:
            print(f'{txt} : not found in vocab')
            

    if current_max_word:
        print(f'Last detected word -> {current_max_word}  \n\n')
        token_lst.append(current_max_word)
        i += skip_
        
    else:
        token_lst.append('<unk>')
        break



เ : not found in vocab
เท : found in vocab
เทอ : found in vocab
เทอก : not found in vocab
เทอกิ : not found in vocab
เทอกิน : not found in vocab
เทอกินข : not found in vocab
เทอกินข้ : not found in vocab
เทอกินข้า : not found in vocab
เทอกินข้าว : not found in vocab
เทอกินข้าวย : not found in vocab
เทอกินข้าวยั : not found in vocab
เทอกินข้าวยัง : not found in vocab
Last detected word -> เทอ  


ก : not found in vocab
กิ : not found in vocab
กิน : found in vocab
กินข : not found in vocab
กินข้ : not found in vocab
กินข้า : not found in vocab
กินข้าว : not found in vocab
กินข้าวย : not found in vocab
กินข้าวยั : not found in vocab
กินข้าวยัง : not found in vocab
Last detected word -> กิน  


ข : not found in vocab
ข้ : not found in vocab
ข้า : found in vocab
ข้าว : not found in vocab
ข้าวย : not found in vocab
ข้าวยั : not found in vocab
ข้าวยัง : not found in vocab
Last detected word -> ข้า  


ว : not found in vocab
วย : not found in vocab
วยั : not found in vocab
วยัง : not found in 

In [ ]:
token_lst

['เทอ', 'กิน', 'ข้า', '<unk>']

# ❌ Maximal Matching + Thai Character Cluster (TCC)

ปัจจุบัน มี tokenizer ไทยที่ถูกใช้อย่างกว้างขวาง อย่าง *newmm* ใช้ concept ของ maximal matching ผสมกับ Thai Character Cluster (TCC)

Thai Character Cluster คืออะไร : https://dl.acm.org/doi/pdf/10.1145/355214.355225

นี่ไม่ใช่ word tokenizer แต่เป็น sub-word tokenizer

ref: https://github.com/wisesight/newmm-tokenizer/blob/main/newmm_tokenizer/newmm.py

In [ ]:
import re
from typing import List, Set

In [ ]:
_RE_TCC = (
    """\
เc็c
เcctาะ
เccีtยะ
เccีtย(?=[เ-ไก-ฮ]|$)
เcc็c
เcิc์c
เcิtc
เcีtยะ?
เcืtอะ?
เc[ิีุู]tย(?=[เ-ไก-ฮ]|$)
เctา?ะ?
cัtวะ
c[ัื]tc[ุิะ]?
c[ิุู]์
c[ะ-ู]t
c็
ct[ะาำ]?
แc็c
แcc์
แctะ
แcc็c
แccc์
โctะ
[เ-ไ]ct
""".replace(
        "c", "[ก-ฮ]"
    )
    .replace("t", "[่-๋]?")
    .split()
)

_PAT_TCC = re.compile("|".join(_RE_TCC))

In [ ]:
# อันนี้ https://github.com/wisesight/newmm-tokenizer/blob/main/newmm_tokenizer/tcc.py

def tcc(text: str) -> str:
    """
    TCC generator, generates Thai Character Clusters
    :param str text: text to be tokenized to character clusters
    :return: subwords (character clusters)
    :rtype: Iterator[str]
    """
    if not text or not isinstance(text, str):
        return ""

    len_text = len(text)
    p = 0
    while p < len_text:
        m = _PAT_TCC.match(text[p:])
        if m:
            n = m.span()[1]
        else:
            n = 1
        yield text[p : p + n]
        p += n


def tcc_pos(text: str) -> Set[int]:
    """
    TCC positions
    :param str text: text to be tokenized to character clusters
    :return: list of the end position of subwords
    :rtype: set[int]
    """
    if not text or not isinstance(text, str):
        return set()

    p_set = set()
    p = 0
    for w in tcc(text):
        p += len(w)
        p_set.add(p)

    return p_set


def segment(text: str) -> List[str]:
    """
    Subword segmentation
    :param str text: text to be tokenized to character clusters
    :return: list of subwords (character clusters), tokenized from the text
    :rtype: list[str]
    """

    return list(tcc(text))

In [ ]:
list(tcc('ชอบเธอนะ'))

['ช', 'อ', 'บ', 'เธ', 'อ', 'นะ']

# ❌ Deepcut

deepcut เป็น algorithm ที่มีความ state-of-art ใน overall

จากปัญหาดังกล่าว จะเห็นได้ว่าการทำ rule based เพื่อสร้าง tokenizer ที่ดีเป็นไปได้ยาก และ ด้วยเหตุผลนี้ state-of-art tokenizer ได้ adopt idea ของ train from data is better than rule based to handle with the above problem

จาก ปัญหา นี้ หากเราสามารถได้ data จากหลายแหล่งเช่น social media, ราชการ etc.

![](https://user-images.githubusercontent.com/1214890/58486992-14c1d880-8191-11e9-9122-8385750e06bd.png)

ใช้ character embedding ในการ represent input

W